
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
import math

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for balck-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.shape

(4870, 65)

In [6]:
df = pd.DataFrame(data[['race','call']])

In [7]:
df['race'].unique()

array(['w', 'b'], dtype=object)

In [8]:
df['call'].unique()

array([ 0.,  1.])

In [9]:
df = pd.DataFrame(data[['race','call']])

In [10]:
df.groupby('race').count()

,call
race,
b,2435
w,2435


In [11]:
# df.groupby('race')
t = pd.pivot_table(df,columns=['race'],aggfunc=np.sum)
t

race,b,w
call,157,235


###Q1. What test is appropriate for this problem? Does CLT apply?

###A1. A hypothesis test for comparing two proportions is an appropriate test for this problem.

The CLT does apply as the following conditions have been met:

*1) Independence:*

    within groups: sampled observations must be independent within each group
        - random sample and n < 10% of population
    between groups: the two groups must be independent of each other


*2) success-failure conditions are met:*
    - nw = nb, so:
    - n*ppool ≥ 10 and n*(1-ppool) ≥ 10 [196 and 2239, respectively]


In [12]:
nw = 2435
nb = 2435
ws = 235.0
bs = 157.0
pw = ws/nw
pb = bs/nb
print pw,pb

0.0965092402464 0.064476386037


In [13]:
# pooled proportion

ppool = (ws+bs)/(nw+nb)
ppool

0.08049281314168377

In [14]:
# success-failure for HT:
print nw*ppool, nw*(1-ppool)

196.0 2239.0


In [15]:
# success-failure for HT:
print nb*ppool, nb*(1-ppool)

196.0 2239.0


In [16]:
# success-failure for CI:
print nw*pw, nw*(1-pw)

235.0 2200.0


In [17]:
# success-failure for CI:
print nb*pb, nb*(1-pb)

157.0 2278.0


###Q2. What are the null and alternate hypotheses?

###A2. The null and alternate hypotheses:

$H_o: p_w = p_b$

$H_a: p_w \ne p_b$


###Q3. Compute margin of error, confidence interval, and p-value.

###A3. 
Point estimate: 0.0320

Margin of error: 0.0153

Confidence interval (95%): [0.0168, 0.0473] 

p-value: ~0.0000

In [18]:
pest = pw - pb
pest

0.032032854209445585

In [19]:
# for CI:
# SE, margin of error, confidence interval; NB: 1.96 (95%) used for CI
SE = math.sqrt((pw*(1-pw)/nw) + (pb*(1-pb)/nb))

ME = 1.96*SE

CI = (pest-ME,pest+ME)
CI

(0.016777447859559147, 0.047288260559332024)

In [20]:
# for HT:
# SE and p-value
poolnum = ppool*(1-ppool)
SEht = math.sqrt((poolnum/nw) + (poolnum/nb))
Z = pest/SEht
Z

4.108412152434346

In [21]:
# p_values = stats.norm.sf(abs(Z)) #one-sided
Z
p_values = stats.norm.sf(abs(Z))*2 #twosided
p_values

3.9838868375850767e-05

In [22]:
# Alternate method for p-values using cdf:
pvalue = -2*stats.norm.cdf(-np.abs(Z))
pvalue

-3.9838868375850767e-05

###Q4. Discuss statistical significance.

###A4. The p-value of the hypothesis test is nearly zero and the 95% CI does not include 0.

We have very strong evidence to indicate that a difference exists between the rate of callbacks on résumés with black-sounding names versus white-sounding names.